In [14]:

# Imports
import os
import requests
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model, model_selection
from tempfile import TemporaryDirectory
import seaborn as sns
import time
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim import lr_scheduler

import torchvision
from torchvision import transforms
from torchvision.utils import make_grid
from torchvision.models import resnet18
import numpy as np
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on device:", DEVICE.upper())

# manual random seed is used for dataset partitioning
# to ensure reproducible results across runs
RNG = torch.Generator().manual_seed(1991)
torch.manual_seed(1991)
torch.cuda.manual_seed(1991)

import random
random.seed(1991)

import numpy as np
np.random.seed(1991)

Running on device: CUDA


In [15]:
# CIFAR 10 dataset
torch.manual_seed(1991)
# Transformations
normalize = transforms.Compose(
    [
        transforms.ToTensor(),
        # transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]
)

# Train data
train_set = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=normalize
)
# Train loader
train_loader = DataLoader(train_set, batch_size=256, shuffle=True, num_workers=2)

# Test data
test_set = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=normalize
)
# Test loader
test_loader = DataLoader(test_set, batch_size=256, shuffle=True, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [16]:
model = resnet18(weights=None, num_classes=10)
for k, m in enumerate(model.modules()):
  print(m)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [17]:
import pandas as pd
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet18
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import torch.nn.utils.prune as prune
from math import sqrt
import json
from copy import deepcopy

def kl_loss_fn(outputs, dist_target):
    kl_loss = F.kl_div(torch.log_softmax(outputs, dim=1), dist_target, log_target=True, reduction='batchmean')
    return kl_loss

def entropy_loss_fn(outputs, labels, dist_target, class_weights):
    ce_loss = F.cross_entropy(outputs, labels, weight=class_weights)
    entropy_dist_target = torch.sum(-torch.exp(dist_target) * dist_target, dim=1)
    entropy_outputs = torch.sum(-torch.softmax(outputs, dim=1) * torch.log_softmax(outputs, dim=1), dim=1)
    entropy_loss = F.mse_loss(entropy_outputs, entropy_dist_target)
    return ce_loss + entropy_loss


def unlearning(
    net,
    retain_loader,
    forget_loader,
    val_loader,
    class_weights=None,
):
    """Simple unlearning by finetuning."""
    epochs = 3.2
    max_iters = int(len(retain_loader) * epochs)
    optimizer = optim.SGD(net.parameters(), lr=0.0005,
                      momentum=0.9, weight_decay=5e-4)
    initial_net = deepcopy(net)

    net.train()
    initial_net.eval()

    def prune_model(net, amount=0.95, rand_init=True):
        # Modules to prune
        modules = list()
        for k, m in enumerate(net.modules()):
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                modules.append((m, 'weight'))
                if m.bias is not None:
                    modules.append((m, 'bias'))

        # Prune criteria
        prune.global_unstructured(
            modules,
            #pruning_method=prune.RandomUnstructured,
            pruning_method=prune.L1Unstructured,
            amount=amount,
        )

        # Perform the prune
        for k, m in enumerate(net.modules()):
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                prune.remove(m, 'weight')
                if m.bias is not None:
                    prune.remove(m, 'bias')

        # Random initialization
        if rand_init:
            for k, m in enumerate(net.modules()):
                if isinstance(m, nn.Conv2d):
                    mask = m.weight == 0
                    c_in = mask.shape[1]
                    k = 1/(c_in*mask.shape[2]*mask.shape[3])
                    randinit = (torch.rand_like(m.weight)-0.5)*2*sqrt(k)
                    m.weight.data[mask] = randinit[mask]
                if isinstance(m, nn.Linear):
                    mask = m.weight == 0
                    c_in = mask.shape[1]
                    k = 1/c_in
                    randinit = (torch.rand_like(m.weight)-0.5)*2*sqrt(k)
                    m.weight.data[mask] = randinit[mask]

    num_iters = 0
    running = True
    prune_amount = 0.99
    prune_model(net, prune_amount, True)
    while running:
        net.train()
        for sample in retain_loader:
            inputs = sample[0]
            targets = sample[1]
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)

            # Get target distribution
            with torch.no_grad():
                original_outputs = initial_net(inputs)
                preds = torch.log_softmax(original_outputs, dim=1)

            optimizer.zero_grad()
            outputs = net(inputs)
            loss = entropy_loss_fn(outputs, targets, preds, class_weights)
            loss.backward()
            optimizer.step()

            num_iters += 1
            # Stop at max iters
            if num_iters > max_iters:
                running = False
                break

    net.eval()

    return net


In [18]:
from sklearn.linear_model import LogisticRegression

def train_mia_on_class(model, class_index, train_set, test_set, no_class=True, splits_num=5):
  model = model.to("cpu")

  if not no_class:
    # Select indencies where class is class_index
    train_class_set = np.where(np.array(train_set.targets) == class_index)[0]
    test_class_set = np.where(np.array(test_set.targets) == class_index)[0]
  else:
    train_class_set = np.arange(len(train_set))
    test_class_set = np.arange(len(test_set))

  max_len = min([len(train_class_set), len(test_class_set)])
  # Make equal sizes
  train_class_set = train_class_set[:max_len]
  test_class_set = test_class_set[:max_len]

  # Obtain subsets
  train_class_set = torch.utils.data.Subset(train_set, train_class_set)
  test_class_set = torch.utils.data.Subset(test_set, test_class_set)


  # Make them
  class_test_loader = torch.utils.data.DataLoader(
    test_class_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )

  class_train_loader = torch.utils.data.DataLoader(
    train_class_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )

  criterion = nn.CrossEntropyLoss(reduction="none")
  # Obtain train and test logits
  logits_train = []
  for i, (images, labels) in enumerate(class_train_loader, 0):
    for img, label in zip(images, labels):
      logits = model(img.unsqueeze(0))
      losses = criterion(logits, label.unsqueeze(0)).numpy(force=True)

      logits_train.append(np.concatenate((logits.detach().numpy()[0], losses)))

  logits_test = []
  for i, (images, labels) in enumerate(class_test_loader, 0):
    for img, label in zip(images, labels):
      logits = model(img.unsqueeze(0))
      losses = criterion(logits, label.unsqueeze(0)).numpy(force=True)

      logits_test.append((np.concatenate((logits.detach().numpy()[0], losses))))

  logits_train, logits_test = np.array(logits_train), np.array(logits_test)
  # Create dataset
  ys = [1] * max_len + [0] * max_len
  ys = np.array(ys)
  p = np.random.permutation(len(ys))
  logits = np.concatenate((logits_train, logits_test))

  logits = logits[p]
  ys = ys[p]

  # Fit logitstic regression
  clf = LogisticRegression(random_state=0, max_iter=1000)
  cv = model_selection.StratifiedShuffleSplit(
        n_splits=splits_num, random_state=0
    )

  clf = model_selection.cross_val_score(
        clf, logits, ys, cv=cv, scoring="accuracy"
    )

  # Output score
  # print(f"Mean MIA attack score for class {class_index} is: " + str(clf.mean()))
  return clf.mean()


In [19]:


# Check accuracy

def accuracy(net, loader):
    """Return accuracy on a dataset given by the data loader."""
    correct = 0
    total = 0
    for i, (inputs, targets) in enumerate(loader, 0):
        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
        outputs = net(inputs)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    return correct / total


In [20]:

def compute_losses(net, loader):
    """Auxiliary function to compute per-sample losses"""

    criterion = nn.CrossEntropyLoss(reduction="none")
    all_losses = []

    for inputs, targets in loader:
        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)

        logits = net(inputs)
        losses = criterion(logits, targets).numpy(force=True)
        for l in losses:
            all_losses.append(l)

    return np.array(all_losses)

## Baseline

In [39]:


print("----- 5 PERCENT -------")
# Generalisation algorithm
# Calculating score across 10 different classes
# Model parameters
model_name = "Resnet18 Baseline"
model_params = "cifar10_resnet18_baseline.pt"

# Percantage of class to forget
amount = 0.05

# Show different classes
classes = np.unique(np.array(train_set.targets))

# mia scores
MIA_scores_before = []
MIA_scores = []

# Accuracy
Accuracy_forget_before = []

Accuracy_retain = []
Accuracy_forget = []
Accuracy_test = []


print(f"Model name: {model_name}")
print(f"Amount: {100 * amount}")

for class_num in classes:
  print(f"------ UNLEARNING CLASS {class_num} ------\n")
  # Define model from trained params

  model_forget_ft = resnet18(weights=None, num_classes=10) # Load resnet18 from pytorch
  model_forget_ft.load_state_dict(torch.load(model_params))
  model_forget_ft.eval()
  model_forget_ft = model_forget_ft.to(DEVICE)

  # --- FORMING DATASET ------
  # Choose random forget indecies from some class
  # Index of class
  class_index = class_num # cars
  class_set = np.where(np.array(train_set.targets) == class_num)[0]

  # Percantage of whole data ( from class )
  amount_int = class_set.shape[0] * amount

  # Get indeces
  forget_idx = np.random.choice(class_set, int(amount_int))

  # construct indices of retain from those of the forget set
  forget_mask = np.zeros(len(train_set.targets), dtype=bool)
  forget_mask[forget_idx] = True
  retain_idx = np.arange(forget_mask.size)[~forget_mask]

  # split train set into a forget and a retain set
  forget_set = torch.utils.data.Subset(train_set, forget_idx)
  retain_set = torch.utils.data.Subset(train_set, retain_idx)

  # Generate forget and retain loaders
  forget_loader = torch.utils.data.DataLoader(
      forget_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  retain_loader = torch.utils.data.DataLoader(
      retain_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  # ---------

  # --- EVALUATING MODEL BEFORE ----
  accuracy_forget = 100.0 * accuracy(model_forget_ft, forget_loader)
  Accuracy_forget_before.append(accuracy_forget)
  print(f"Accuracy before on forget set: {accuracy_forget}")

  mia_attack = train_mia_on_class(model_forget_ft, class_num, forget_set, test_set, True, 2)
  MIA_scores_before.append(mia_attack)
  print(f"MIA score before on forget set: {np.round(mia_attack, 3)}")
  print("\n")

  model_forget_ft = model_forget_ft.to(DEVICE)
  # --------------

  # Unlearn
  model_ft_forget = unlearning(model_forget_ft, retain_loader, forget_loader, test_loader)

  # Compare accuracy
  print(f"Retain set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, retain_loader):0.1f}%")
  print(f"Test set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, test_loader):0.1f}%")
  print(f"Forget set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, forget_loader):0.1f}%")

  Accuracy_retain.append(100.0 * accuracy(model_ft_forget, retain_loader))
  Accuracy_forget.append(100.0 * accuracy(model_ft_forget, forget_loader))
  Accuracy_test.append(100.0 * accuracy(model_ft_forget, test_loader))
  print()
  model_ft_forget.eval()
  mia_scores_fr = train_mia_on_class(model_forget_ft, class_num, forget_set, test_set, True, 5)
  MIA_scores.append(mia_scores_fr)
  print(
      f"The MIA has an accuracy of {mia_scores_fr:.3f} on forgotten vs unseen images on FORGET model"
  )
  print()
  print('-' * 20)

----- 5 PERCENT -------
Model name: Resnet18 Baseline
Amount: 5.0
------ UNLEARNING CLASS 0 ------

Accuracy before on forget set: 97.6
MIA score before on forget set: 0.94


Retain set accuracy FORGET model: 68.1%
Test set accuracy FORGET model: 58.2%
Forget set accuracy FORGET model: 52.8%

The MIA has an accuracy of 0.856 on forgotten vs unseen images on FORGET model

--------------------
------ UNLEARNING CLASS 1 ------

Accuracy before on forget set: 96.8
MIA score before on forget set: 0.95


Retain set accuracy FORGET model: 68.6%
Test set accuracy FORGET model: 57.5%
Forget set accuracy FORGET model: 57.6%

The MIA has an accuracy of 0.896 on forgotten vs unseen images on FORGET model

--------------------
------ UNLEARNING CLASS 2 ------

Accuracy before on forget set: 93.2
MIA score before on forget set: 0.89


Retain set accuracy FORGET model: 67.5%
Test set accuracy FORGET model: 57.2%
Forget set accuracy FORGET model: 56.4%

The MIA has an accuracy of 0.752 on forgotten vs

In [40]:

# mia scores
print(MIA_scores_before)
print(MIA_scores)

# Accuracy
print(Accuracy_forget_before)

print(Accuracy_retain)
print(Accuracy_forget)
print(Accuracy_test)


[0.94, 0.95, 0.89, 0.85, 0.9, 0.87, 0.9299999999999999, 0.95, 0.91, 0.89]
[0.8560000000000001, 0.8960000000000001, 0.752, 0.792, 0.8320000000000001, 0.808, 0.8480000000000001, 0.8280000000000001, 0.82, 0.804]
[97.6, 96.8, 93.2, 85.2, 94.8, 88.4, 96.8, 95.19999999999999, 98.4, 95.6]
[68.08891211286854, 68.57527584058525, 67.53285903774268, 67.76605366294845, 67.00164810869478, 67.66627138062027, 64.02701290348514, 63.402134373053045, 66.63317188850259, 66.67872575620541]
[52.800000000000004, 57.599999999999994, 56.39999999999999, 28.799999999999997, 48.4, 51.6, 79.60000000000001, 50.8, 72.39999999999999, 72.39999999999999]
[58.160000000000004, 57.48, 57.16, 56.34, 56.330000000000005, 56.98, 55.26, 52.980000000000004, 56.44, 55.2]


In [41]:

mean_mia_score_before_baseline = np.mean(np.array(MIA_scores_before))
mean_mia_score_after_baseline = np.mean(np.array(MIA_scores))

mean_accuracy_forget_before_baseline = np.mean(np.array(Accuracy_forget_before))

mean_accuracy_forget_after_baseline = np.mean(np.array(Accuracy_forget))
mean_accuracy_retain_baseline = np.mean(np.array(Accuracy_retain))
mean_accuracy_test_baseline = np.mean(np.array(Accuracy_test))

In [42]:


print(mean_mia_score_before_baseline)
print(mean_mia_score_after_baseline)

print(mean_accuracy_forget_before_baseline)

print(mean_accuracy_forget_after_baseline)
print(mean_accuracy_retain_baseline)
print(mean_accuracy_test_baseline)

0.908
0.8236000000000001
94.2
57.08
66.73720650647061
56.233000000000004


## Dropout model

In [43]:
def modify_model(model_train_reg):
  from collections import OrderedDict

  layer1 = model_train_reg.layer1

  for indx in range(len(layer1)):
    modified_layers = OrderedDict()
    for name, feature in layer1[indx].named_children():
      modified_layers[name] = feature
      if isinstance(feature, nn.ReLU):
        modified_layers["dropout"] = nn.Dropout(p=0.3)

    model_train_reg.layer1[indx] = nn.Sequential(modified_layers)



  layer2 = model_train_reg.layer2

  for indx in [1]:
    modified_layers = OrderedDict()
    for name, feature in layer2[indx].named_children():
      modified_layers[name] = feature
      if isinstance(feature, nn.ReLU):
        modified_layers["dropout"] = nn.Dropout(p=0.3)

    model_train_reg.layer2[indx] = nn.Sequential(modified_layers)

  layer3 = model_train_reg.layer3

  for indx in [1]:
    modified_layers = OrderedDict()
    for name, feature in layer3[indx].named_children():
      modified_layers[name] = feature
      if isinstance(feature, nn.ReLU):
        modified_layers["dropout"] = nn.Dropout(p=0.3)
    model_train_reg.layer3[indx] = nn.Sequential(modified_layers)

  layer4 = model_train_reg.layer4

  for indx in [1]:
    modified_layers = OrderedDict()
    for name, feature in layer4[indx].named_children():
      modified_layers[name] = feature
      if isinstance(feature, nn.ReLU):
        modified_layers["dropout"] = nn.Dropout(p=0.3)

    model_train_reg.layer4[indx] = nn.Sequential(modified_layers)

  return model_train_reg

In [44]:


print("----- 5 PERCENT DROPOUT -------")
# Generalisation algorithm
# Calculating score across 10 different classes
# Model parameters
model_name = "Resnet18 Dropout"
model_params = "cifar10_resnet18_l2_dropout_regularization.pt"

# Percantage of class to forget
amount = 0.05

# Show different classes
classes = np.unique(np.array(train_set.targets))

# mia scores
MIA_scores_before = []
MIA_scores = []

# Accuracy
Accuracy_forget_before = []

Accuracy_retain = []
Accuracy_forget = []
Accuracy_test = []


print(f"Model name: {model_name}")
print(f"Amount: {100 * amount}")

for class_num in classes:
  print(f"------ UNLEARNING CLASS {class_num} ------\n")
  # Define model from trained params

  model_forget_ft = resnet18(weights=None, num_classes=10) # Load resnet18 from pytorch
  model_forget_ft = model_forget_ft.to(DEVICE)
  model_forget_ft = modify_model(model_forget_ft)

  model_forget_ft.load_state_dict(torch.load(model_params))
  model_forget_ft.eval()

  # --- FORMING DATASET ------
  # Choose random forget indecies from some class
  # Index of class
  class_index = class_num # cars
  class_set = np.where(np.array(train_set.targets) == class_num)[0]

  # Percantage of whole data ( from class )
  amount_int = class_set.shape[0] * amount

  # Get indeces
  forget_idx = np.random.choice(class_set, int(amount_int))

  # construct indices of retain from those of the forget set
  forget_mask = np.zeros(len(train_set.targets), dtype=bool)
  forget_mask[forget_idx] = True
  retain_idx = np.arange(forget_mask.size)[~forget_mask]

  # split train set into a forget and a retain set
  forget_set = torch.utils.data.Subset(train_set, forget_idx)
  retain_set = torch.utils.data.Subset(train_set, retain_idx)

  # Generate forget and retain loaders
  forget_loader = torch.utils.data.DataLoader(
      forget_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  retain_loader = torch.utils.data.DataLoader(
      retain_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  # ---------

  # --- EVALUATING MODEL BEFORE ----
  accuracy_forget = 100.0 * accuracy(model_forget_ft, forget_loader)
  Accuracy_forget_before.append(accuracy_forget)
  print(f"Accuracy before on forget set: {accuracy_forget}")

  mia_attack = train_mia_on_class(model_forget_ft, class_num, forget_set, test_set, True, 2)
  MIA_scores_before.append(mia_attack)
  print(f"MIA score before on forget set: {np.round(mia_attack, 3)}")
  print("\n")

  model_forget_ft = model_forget_ft.to(DEVICE)
  # --------------

  # Unlearn
  model_ft_forget = unlearning(model_forget_ft, retain_loader, forget_loader, test_loader)

  # Compare accuracy
  print(f"Retain set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, retain_loader):0.1f}%")
  print(f"Test set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, test_loader):0.1f}%")
  print(f"Forget set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, forget_loader):0.1f}%")

  Accuracy_retain.append(100.0 * accuracy(model_ft_forget, retain_loader))
  Accuracy_forget.append(100.0 * accuracy(model_ft_forget, forget_loader))
  Accuracy_test.append(100.0 * accuracy(model_ft_forget, test_loader))
  print()
  model_ft_forget.eval()
  mia_scores_fr = train_mia_on_class(model_forget_ft, class_num, forget_set, test_set, True, 5)
  MIA_scores.append(mia_scores_fr)
  print(
      f"The MIA has an accuracy of {mia_scores_fr:.3f} on forgotten vs unseen images on FORGET model"
  )
  print()
  print('-' * 20)

----- 5 PERCENT DROPOUT -------
Model name: Resnet18 Dropout
Amount: 5.0
------ UNLEARNING CLASS 0 ------

Accuracy before on forget set: 84.0
MIA score before on forget set: 0.93


Retain set accuracy FORGET model: 60.9%
Test set accuracy FORGET model: 59.2%
Forget set accuracy FORGET model: 60.8%

The MIA has an accuracy of 0.888 on forgotten vs unseen images on FORGET model

--------------------
------ UNLEARNING CLASS 1 ------

Accuracy before on forget set: 87.6
MIA score before on forget set: 0.89


Retain set accuracy FORGET model: 67.3%
Test set accuracy FORGET model: 65.9%
Forget set accuracy FORGET model: 80.8%

The MIA has an accuracy of 0.892 on forgotten vs unseen images on FORGET model

--------------------
------ UNLEARNING CLASS 2 ------

Accuracy before on forget set: 69.6
MIA score before on forget set: 0.81


Retain set accuracy FORGET model: 62.5%
Test set accuracy FORGET model: 60.9%
Forget set accuracy FORGET model: 33.6%

The MIA has an accuracy of 0.844 on forgo

In [45]:

# mia scores
print(MIA_scores_before)
print(MIA_scores)

# Accuracy
print(Accuracy_forget_before)

print(Accuracy_retain)
print(Accuracy_forget)
print(Accuracy_test)

[0.9299999999999999, 0.89, 0.81, 0.89, 0.9, 0.81, 0.87, 0.9099999999999999, 0.9, 0.85]
[0.8879999999999999, 0.892, 0.844, 0.8240000000000001, 0.8559999999999999, 0.82, 0.8720000000000001, 0.9040000000000001, 0.792, 0.8719999999999999]
[84.0, 87.6, 69.6, 60.0, 76.4, 70.39999999999999, 85.2, 84.39999999999999, 87.6, 87.2]
[60.94060898402171, 67.32585714859923, 62.47814290021103, 63.613891233570484, 61.90208216094541, 65.65853462481411, 60.748095592225596, 64.93488222525926, 56.10667845730249, 57.11184805547182]
[60.8, 80.80000000000001, 33.6, 36.8, 77.2, 45.2, 84.39999999999999, 60.4, 74.8, 59.599999999999994]
[59.209999999999994, 65.94, 60.919999999999995, 61.980000000000004, 61.47, 64.28, 59.730000000000004, 63.13999999999999, 55.22, 55.769999999999996]


In [46]:


mean_mia_score_before_dropout = np.mean(np.array(MIA_scores_before))
mean_mia_score_after_dropout = np.mean(np.array(MIA_scores))

mean_accuracy_forget_before_dropout = np.mean(np.array(Accuracy_forget_before))

mean_accuracy_forget_after_dropout = np.mean(np.array(Accuracy_forget))
mean_accuracy_retain_dropout = np.mean(np.array(Accuracy_retain))
mean_accuracy_test_dropout = np.mean(np.array(Accuracy_test))

In [47]:


print(mean_mia_score_before_dropout)
print(mean_mia_score_after_dropout)

print(mean_accuracy_forget_before_dropout)

print(mean_accuracy_forget_after_dropout)
print(mean_accuracy_retain_dropout)
print(mean_accuracy_test_dropout)


0.876
0.8564
79.24
61.36
62.08206213824211
60.766


## Augmentation model

In [48]:

# CIFAR 10 dataset
torch.manual_seed(1991)
# Transformations
normalize = transforms.Compose(
    [
         transforms.RandomRotation(30), # Randomly rotate some images by 20 degrees
         transforms.RandomHorizontalFlip(), # Randomly horizontal flip the images
         transforms.ColorJitter(brightness = 0.1, # Randomly adjust color jitter of the images
                                                            contrast = 0.1,
                                                            saturation = 0.1),
         transforms.RandomAdjustSharpness(sharpness_factor = 2,
                                                                      p = 0.1), # Randomly adjust sharpness
          transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)), #Performs actions like zooms, change shear angles.
        transforms.ToTensor(),
    ]
)

to_tensor_transform = transforms.Compose(
    [
        transforms.ToTensor(),
    ]
)


# Train data
train_set = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=normalize
)
# Train loader
train_loader = DataLoader(train_set, batch_size=256, shuffle=True, num_workers=2)

# Test data
test_set = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=to_tensor_transform
)
# Test loader
test_loader = DataLoader(test_set, batch_size=256, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [49]:


print("----- 5 PERCENT -------")
# Generalisation algorithm
# Calculating score across 10 different classes
# Model parameters
model_name = "Resnet18 Augmentation"
model_params = "cifar10_resnet18_augmentation.pt"

# Percantage of class to forget
amount = 0.05

# Show different classes
classes = np.unique(np.array(train_set.targets))

# mia scores
MIA_scores_before = []
MIA_scores = []

# Accuracy
Accuracy_forget_before = []

Accuracy_retain = []
Accuracy_forget = []
Accuracy_test = []


print(f"Model name: {model_name}")
print(f"Amount: {100 * amount}")

for class_num in classes:
  print(f"------ UNLEARNING CLASS {class_num} ------\n")
  # Define model from trained params

  model_forget_ft = resnet18(weights=None, num_classes=10) # Load resnet18 from pytorch
  model_forget_ft.load_state_dict(torch.load(model_params))
  model_forget_ft.eval()
  model_forget_ft = model_forget_ft.to(DEVICE)

  # --- FORMING DATASET ------
  # Choose random forget indecies from some class
  # Index of class
  class_index = class_num # cars
  class_set = np.where(np.array(train_set.targets) == class_num)[0]

  # Percantage of whole data ( from class )
  amount_int = class_set.shape[0] * amount

  # Get indeces
  forget_idx = np.random.choice(class_set, int(amount_int))

  # construct indices of retain from those of the forget set
  forget_mask = np.zeros(len(train_set.targets), dtype=bool)
  forget_mask[forget_idx] = True
  retain_idx = np.arange(forget_mask.size)[~forget_mask]

  # split train set into a forget and a retain set
  forget_set = torch.utils.data.Subset(train_set, forget_idx)
  retain_set = torch.utils.data.Subset(train_set, retain_idx)

  # Generate forget and retain loaders
  forget_loader = torch.utils.data.DataLoader(
      forget_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  retain_loader = torch.utils.data.DataLoader(
      retain_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  # ---------

  # --- EVALUATING MODEL BEFORE ----
  accuracy_forget = 100.0 * accuracy(model_forget_ft, forget_loader)
  Accuracy_forget_before.append(accuracy_forget)
  print(f"Accuracy before on forget set: {accuracy_forget}")

  mia_attack = train_mia_on_class(model_forget_ft, class_num, forget_set, test_set, True, 2)
  MIA_scores_before.append(mia_attack)
  print(f"MIA score before on forget set: {np.round(mia_attack, 3)}")
  print("\n")

  model_forget_ft = model_forget_ft.to(DEVICE)
  # --------------

  # Unlearn
  model_ft_forget = unlearning(model_forget_ft, retain_loader, forget_loader, test_loader)

  # Compare accuracy
  print(f"Retain set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, retain_loader):0.1f}%")
  print(f"Test set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, test_loader):0.1f}%")
  print(f"Forget set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, forget_loader):0.1f}%")

  Accuracy_retain.append(100.0 * accuracy(model_ft_forget, retain_loader))
  Accuracy_forget.append(100.0 * accuracy(model_ft_forget, forget_loader))
  Accuracy_test.append(100.0 * accuracy(model_ft_forget, test_loader))
  print()
  model_forget_ft.eval()
  mia_scores_fr = train_mia_on_class(model_forget_ft, class_num, forget_set, test_set, True, 5)
  MIA_scores.append(mia_scores_fr)
  print(
      f"The MIA has an accuracy of {mia_scores_fr:.3f} on forgotten vs unseen images on FORGET model"
  )
  print()
  print('-' * 20)


----- 5 PERCENT -------
Model name: Resnet18 Augmentation
Amount: 5.0
------ UNLEARNING CLASS 0 ------

Accuracy before on forget set: 81.2
MIA score before on forget set: 0.87


Retain set accuracy FORGET model: 51.4%
Test set accuracy FORGET model: 55.4%
Forget set accuracy FORGET model: 46.4%

The MIA has an accuracy of 0.808 on forgotten vs unseen images on FORGET model

--------------------
------ UNLEARNING CLASS 1 ------

Accuracy before on forget set: 78.8
MIA score before on forget set: 0.92


Retain set accuracy FORGET model: 52.4%
Test set accuracy FORGET model: 55.8%
Forget set accuracy FORGET model: 64.8%

The MIA has an accuracy of 0.844 on forgotten vs unseen images on FORGET model

--------------------
------ UNLEARNING CLASS 2 ------

Accuracy before on forget set: 73.6
MIA score before on forget set: 0.82


Retain set accuracy FORGET model: 50.9%
Test set accuracy FORGET model: 54.4%
Forget set accuracy FORGET model: 40.8%

The MIA has an accuracy of 0.756 on forgotte

In [50]:

# mia scores
print(MIA_scores_before)
print(MIA_scores)

# Accuracy
print(Accuracy_forget_before)

print(Accuracy_retain)
print(Accuracy_forget)
print(Accuracy_test)

[0.87, 0.9199999999999999, 0.82, 0.81, 0.91, 0.84, 0.8300000000000001, 0.91, 0.8200000000000001, 0.84]
[0.808, 0.844, 0.756, 0.788, 0.8, 0.852, 0.8640000000000001, 0.8240000000000001, 0.784, 0.808]
[81.2, 78.8, 73.6, 65.2, 76.8, 68.8, 82.0, 81.2, 89.2, 77.60000000000001]
[51.30648014150185, 52.39674404582454, 51.068062616803644, 51.25097968288419, 51.75955141990072, 51.28828683977812, 49.40809968847352, 52.68795594766777, 51.545480123799194, 52.78654259702152]
[50.0, 62.8, 38.800000000000004, 28.000000000000004, 48.8, 30.0, 60.0, 62.8, 66.4, 45.6]
[55.400000000000006, 55.800000000000004, 54.44, 54.82, 54.510000000000005, 55.26, 52.75, 56.26, 55.11000000000001, 57.05]


In [51]:


mean_mia_score_before_aug = np.mean(np.array(MIA_scores_before))
mean_mia_score_after_aug = np.mean(np.array(MIA_scores))

mean_accuracy_forget_before_aug = np.mean(np.array(Accuracy_forget_before))

mean_accuracy_forget_after_aug = np.mean(np.array(Accuracy_forget))
mean_accuracy_retain_aug = np.mean(np.array(Accuracy_retain))
mean_accuracy_test_aug = np.mean(np.array(Accuracy_test))

In [52]:


print(mean_mia_score_before_aug)
print(mean_mia_score_after_aug)

print(mean_accuracy_forget_before_aug)

print(mean_accuracy_forget_after_aug)
print(mean_accuracy_retain_aug)
print(mean_accuracy_test_aug)


0.857
0.8128
77.44
49.32000000000001
51.5498183103655
55.14
